*Note: Turn on GPU for this notebook.*

## Dependencies and Libraries

In [ ]:
# download the required pacakges
!pip install tensorflow_text

In [16]:
# import the libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

## Load the data

In [3]:
# Load the data
### data_path is the path of the csv file

data_path = "/content/drive/Shareddrives/team MAMI/MAMI/TRAIN/training.csv"
df = pd.read_csv(data_path, delimiter="\t")
df.head(5)

,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription
0,1.jpg,0,0,0,0,0,Milk Milk.zip
1,10.jpg,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,0,0,0,0,MAN SEEKING WOMAN Ignad 18 O
4,10006.jpg,0,0,0,0,0,Me explaining the deep lore of. J.R.R. Tolkein...


## Data Summary and Preprocessing

In [17]:
# checking if the classes are balanced
df['misogynous'].value_counts()

1    5000
0    5000
Name: misogynous, dtype: int64

The classes are evenly distributed (i.e. balanced class).

In [18]:
# splitting the data into train and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Text Transcription'],df['misogynous'], stratify=df['misogynous'])

## BERT Implementation

In [6]:
# downloading the required packages for BERT
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Functional model below.

In [11]:
# initializing BERT layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# initializing NN layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

## Model Training

In [13]:
# accuracy metrics
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [14]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
235/235 [==============================] - 159s 663ms/step - loss: 0.6944 - accuracy: 0.5455 - precision: 0.5444 - recall: 0.5571
Epoch 2/5
235/235 [==============================] - 156s 665ms/step - loss: 0.6632 - accuracy: 0.6044 - precision: 0.6037 - recall: 0.6077
Epoch 3/5
235/235 [==============================] - 155s 661ms/step - loss: 0.6491 - accuracy: 0.6188 - precision: 0.6210 - recall: 0.6096
Epoch 4/5
235/235 [==============================] - 155s 659ms/step - loss: 0.6396 - accuracy: 0.6385 - precision: 0.6392 - recall: 0.6360
Epoch 5/5
235/235 [==============================] - 155s 661ms/step - loss: 0.6331 - accuracy: 0.6445 - precision: 0.6464 - recall: 0.6381


In [15]:
model.evaluate(X_test, y_test)

79/79 [==============================] - 53s 659ms/step - loss: 0.6145 - accuracy: 0.6808 - precision: 0.7678 - recall: 0.5184


[0.6144709587097168,
 0.6808000206947327,
 0.7677724957466125,
 0.5184000134468079]